In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from lpfm.utils.plotting.plot_lossVsTime import LossVsTimePlotter
from lpfm.utils.plotting.base_plotter import calculate_combined_stats, calculate_combined_stats_rollout

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect display width
pd.set_option('display.float_format', lambda x: '%.6f' % x)  # Format float numbers

base_dir = Path("/scratch/zsa8rk/logs")

# Get Mean, Median and STD from loss dataframes
RUNS = [
    ("fno-s-main", "best_model"),
    ("fno-m-main", "best_model"),
    ("ti-main-4-1", "0080"),
    ("m-main-4-1", "0141"),
    ("l-main-4-1", "0140"),
    ("xl-main-4-1-a", "0080"),
]

In [ ]:


for run, checkpoint in RUNS:
    eval_dir = base_dir / run / "eval" / checkpoint
    loss_df = pd.read_csv(eval_dir / "losses.csv", header=0)
    # Example usage:
    # Calculate combined means for different flow types
    flow_patterns = [
        'cylinder_sym_flow_water',
        'cylinder_pipe_flow_water',
        'object_periodic_flow_water',
        'object_sym_flow_water',
        'object_sym_flow_air',
        'rayleigh_benard',
        'rayleigh_benard_obstacle',
        'twophase_flow',
        'shear_flow',
        'euler_multi_quadrants_periodicBC',
        'heated_object_pipe_flow_air',
        'cooled_object_pipe_flow_air',
        'acoustic_scattering_inclusions'

    ]

    combined_means = calculate_combined_stats(loss_df, flow_patterns)

    print(f"Run: {run}, Checkpoint: {checkpoint}")
    display(combined_means)

Run: fno-s-main, Checkpoint: best_model


,Combined Mean,Combined Median,Combined Std
Dataset,,,
cylinder_sym_flow_water,0.000218,0.000208,0.000055
cylinder_pipe_flow_water,0.000332,0.000286,0.000155
object_periodic_flow_water,0.010790,0.001448,0.041348
object_sym_flow_water,0.010961,0.001650,0.056004
object_sym_flow_air,0.009987,0.002961,0.014698
rayleigh_benard,0.258713,0.015768,25.576512
rayleigh_benard_obstacle,0.005441,0.003001,0.006136
twophase_flow,0.026762,0.002419,0.211623
shear_flow,0.012376,0.000763,0.071838


Run: fno-m-main, Checkpoint: best_model


,Combined Mean,Combined Median,Combined Std
Dataset,,,
cylinder_sym_flow_water,0.000137,0.000126,0.000058
cylinder_pipe_flow_water,0.000196,0.000162,0.000159
object_periodic_flow_water,0.010807,0.001212,0.043467
object_sym_flow_water,0.010986,0.001605,0.059302
object_sym_flow_air,0.009621,0.002567,0.014726
rayleigh_benard,0.248081,0.015030,25.574181
rayleigh_benard_obstacle,0.005318,0.002958,0.006096
twophase_flow,0.025776,0.001445,0.205577
shear_flow,0.012056,0.000596,0.071788


Run: ti-main-4-1, Checkpoint: 0080


...

,Combined Mean,Combined Median,Combined Std
Dataset,,,
cylinder_sym_flow_water,0.000012,0.000011,0.000003
cylinder_pipe_flow_water,0.000015,0.000013,0.000005
object_periodic_flow_water,0.000783,0.000073,0.005036
object_sym_flow_water,0.000828,0.000081,0.008357
object_sym_flow_air,0.000341,0.000122,0.000552
rayleigh_benard,0.197206,0.000823,25.714951
rayleigh_benard_obstacle,0.000382,0.000181,0.000475
twophase_flow,0.007825,0.000080,0.082347
shear_flow,0.000401,0.000046,0.002123


Run: l-main-4-1, Checkpoint: 0140


,Combined Mean,Combined Median,Combined Std
Dataset,,,
cylinder_sym_flow_water,0.000007,0.000006,0.000003
cylinder_pipe_flow_water,0.000008,0.000007,0.000004
object_periodic_flow_water,0.000629,0.000051,0.004471
object_sym_flow_water,0.000647,0.000050,0.006496
object_sym_flow_air,0.000255,0.000087,0.000435
rayleigh_benard,0.197048,0.000695,25.771786
rayleigh_benard_obstacle,0.000309,0.000137,0.000397
twophase_flow,0.007012,0.000046,0.079488
shear_flow,0.000255,0.000025,0.001652


Run: xl-main-4-1-a, Checkpoint: 0080


,Combined Mean,Combined Median,Combined Std
Dataset,,,
cylinder_sym_flow_water,0.000006,0.000005,0.000001
cylinder_pipe_flow_water,0.000006,0.000005,0.000002
object_periodic_flow_water,0.000542,0.000038,0.003823
object_sym_flow_water,0.000516,0.000034,0.006197
object_sym_flow_air,0.000189,0.000063,0.000337
rayleigh_benard,0.213368,0.000515,26.839195
rayleigh_benard_obstacle,0.000227,0.000095,0.000314
twophase_flow,0.005722,0.000031,0.062175
shear_flow,0.000198,0.000019,0.001364


In [ ]:
for run in RUNS:
    eval_dir = base_dir / run / "eval"
    rollout_df = pd.read_csv(eval_dir / "rollout_losses.csv", header=[0,1,2])
    single_step_df = pd.read_csv(eval_dir / "single_step_losses.csv", header=[0,1,2])


    # Calculate combined means for different flow types
    flow_patterns = [
        'cylinder_sym_flow_water',
        'cylinder_pipe_flow_water',
        'object_periodic_flow_water',
        'object_sym_flow_water',
        'object_sym_flow_air',
        'rayleigh_benard',
        'rayleigh_benard_obstacle',
        'twophase_flow',
        'shear_flow',
        'euler_multi_quadrants_periodicBC',
        'heated_object_pipe_flow_air',
        'cooled_object_pipe_flow_air',
        'acoustic_scattering_inclusions'

    ]
    combined_df = calculate_combined_stats_rollout(rollout_df, column_patterns=flow_patterns)

    # combined_means = calculate_combined_stats(rollout_df, flow_patterns, level=0)
    # combined_means = calculate_combined_stats(single_step_df, flow_patterns, level=0)
    #print(combined_df.head())
    # display(combined_df)

    data_mean = combined_df["cylinder_sym_flow_water"]["mean"]
    data_std = combined_df["cylinder_sym_flow_water"]["std"]
    # Plot the combined dfs over time
    x_ticks = [0, 25, 50]
    y_ticks = [0, 0.2]
    plotter = LossVsTimePlotter(x_ticks=x_ticks, y_ticks=y_ticks)
    plotter.plot(mean_loss=data_mean.values, std_loss=None)
    plotter.legend("cylinder_sym_flow_water")